# PV Simulation

The OpenEnergyID Module "PV Simulation" is a tool that helps you simulate a PV plant.

## PVLib implementation

A future version of the OpenEnergyID Module "PV Simulation" will use the PVLib library to simulate a PV plant. The PVLib library is a Python library that provides a set of functions to simulate a PV plant. The library is based on the Sandia PV performance model and the CEC performance model.

## Elia implementation

This is a simplified model, that uses the measured load factors for all the PV plants in Belgium. The load factors are calculated and published by Elia, the Belgian transmission system operator.

### Imports

In [ ]:
import datetime as dt
import aiohttp

from openenergyid import elia
from openenergyid.pvsim import LoadFactorPVSimulator

### Parameters

In [ ]:
start = dt.date(2025, 1, 1)
end = dt.date(2025, 2, 1)

panel_power = 5  # kWp
inverter_power = 3  # kWp

### Get Elia Load Factors

PV Production is defined as Elia dataset ods032. Check the Elia documentation for more info.

In [ ]:
async with aiohttp.ClientSession() as session:
    pv_load_factor_json = await elia.get_dataset(
        dataset="ods032",
        start=start,
        end=end,
        region=elia.Region.Antwerp,
        select={"loadfactor"},
        session=session,
    )

pv_load_factors = elia.parse_response(
    data=pv_load_factor_json, index="datetime", columns=["loadfactor"]
)

In [ ]:
# As you can see, load factor is a number between 0 and 100

pv_load_factors.plot()

### Simulation

In [ ]:
simulator = LoadFactorPVSimulator(
    load_factors=pv_load_factors["loadfactor"],
    panel_power=panel_power,
    inverter_power=inverter_power,
)

In [ ]:
simulator.simulation_results

In [ ]:
simulator.simulation_results.plot()